# **Purpose**
This notebook is a tutorial for ZapMeNot.

# **Configure for the Tutorial**
This step won't be required for production work.  The module search path is being altered to preferentially run
the "nightly builds" of both WatpropPy and ZapMeNot.

In [ ]:
import sys
sys.path.insert(0,'/naf/rad/workshop/packages/WatpropPy')
sys.path.insert(0,'/naf/rad/workshop/packages/ZapMeNot')

# **Import packages**
The Python packages required by this notebook are imported. The package and Python interpreter versions are printed to ensure analysis reproducibility. 

In [ ]:
from IPython.display import display, Markdown, Math, Latex, Javascript, HTML
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import math
from zap_me_not import model,source,shield,detector,material
import pyvista as pv
import WatpropPy

# **Problem Descripton**
Modified version of ESIS benchmark shielding problem \#1.

The source is a water-filled tank 6 ft high and 7 ft in diameter.
The tank wall is steel with a thickness of 1 inch.
Uniformly distributed in the water is a source containing
10 Ci of Co<sup>60</sup>, 4,300 Bq of Cr<sup>48</sup>, and an unknown
isotope with a 0.4 MeV photon with an intensity of 3.0E+4 photons/sec/cm<sup>3</sup>.
  
The  detector location is 20 ft from the tank centerline
at the vertical centerline of the tank.
A concrete shield wall between tank and detector is located 10 ft
from the tank wall and is 6 in thick.
The water has a temperature of 76 degF at a pressure of 14.7 psia. 
The air density is 0.00122 g/cm3.
The the steel density is 7.8 g/cm<sup>3</sup>.
The concrete density is 2.4 g/cm<sup>3</sup>.  
The buildup material is concrete.

Update: Water temperature changed to 170 degF.

Update: Add 125 curies of I-131 to the source.

Update: Add 300 curies of Cs-137 and daughters to the source.

Update: Change source modeling to 10, 10, 10.


# **Conversion of Units to CGS**

In [ ]:
def in_to_cm(length):
    return length*2.54

def ft_to_cm(length):
    return length*12*2.54

english = WatpropPy.units.english
si = WatpropPy.units.si

tank_height_cm = ft_to_cm(6)
tank_inner_radius_cm = ft_to_cm(7/2)
tank_wall_thickness_cm = in_to_cm(1)
tank_outer_radius_cm = tank_inner_radius_cm + tank_wall_thickness_cm

detector_distance_cm = ft_to_cm(20)

concrete_shield_thickness_cm = in_to_cm(6)
concrete_shield_inner_edge_cm = ft_to_cm(10) + tank_outer_radius_cm
concrete_shield_outer_edge_cm = concrete_shield_inner_edge_cm + concrete_shield_thickness_cm

water_density_mks = 1/(WatpropPy.v_pt(14.7, 170, english, si))
water_density_cgs = water_density_mks / 1000

display(Markdown(f"**tank height = {tank_height_cm:2.4} cm**"))
display(Markdown(f"**tank inner radius = {tank_inner_radius_cm:2.4} cm**"))
display(Markdown(f"**tank wall thickness = {tank_wall_thickness_cm:2.4} cm**"))
display(Markdown(f"**tank outer radius = {tank_outer_radius_cm:2.4} cm**"))
display(Markdown(f"**detector location = {detector_distance_cm:2.4} cm**"))
display(Markdown(f"**concrete shield inner edge = {concrete_shield_inner_edge_cm:2.4} cm**"))
display(Markdown(f"**concrete shield thickness = {concrete_shield_thickness_cm:2.4} cm**"))
display(Markdown(f"**concrete shield outer edge = {concrete_shield_outer_edge_cm:2.4} cm**"))
display(Markdown(f"**water density = {water_density_cgs:2.4} g/cc**"))

# **Model Construction**
The centerline of the tank will be on the Z axis.  The dose point will be located along
and above the X axis.

In [ ]:
# build the source, including dimensions, materials, and source composition
my_source = source.ZAlignedCylinderSource(material_name="water", density=water_density_cgs, \
    cylinder_radius=tank_inner_radius_cm, \
    cylinder_center=[0, 0, tank_height_cm/2], cylinder_length=tank_height_cm)
my_source.add_isotope_curies('Co-60',10)
my_source.add_isotope_bq('Cr-48',4300)
my_source.add_photon(0.4,3.0E+4)
my_source.add_isotope_curies('I-131',125)
my_source.add_isotope_curies('Cs-137',300)
my_source.include_key_progeny = True
my_source.points_per_dimension = [10,10,10]

# create the overall geometry
tank_model = model.Model()
tank_model.add_source(my_source)

# add the tank wall
tank_model.add_shield(shield.ZAlignedInfiniteAnnulus("iron", \
    cylinder_inner_radius=tank_inner_radius_cm, \
    cylinder_center=[0, 0, tank_height_cm/2], \
    cylinder_outer_radius=tank_outer_radius_cm, density=7.8))

# add the concrete shield
tank_model.add_shield(shield.SemiInfiniteXSlab("concrete", x_start=concrete_shield_inner_edge_cm, \
       x_end=concrete_shield_outer_edge_cm, density=2.4))

# add the air, buildup factor, and detector
tank_model.set_filler_material('air',density=0.00122) # g/cc
tank_model.set_buildup_factor_material(material.Material('concrete'))
tank_model.add_detector(detector.Detector(detector_distance_cm, 0, tank_height_cm/2))

# **Calculate Dose Rate**

In [ ]:
result = tank_model.calculate_exposure()
display(Markdown(f"**dose rate = {result:2.3} mR/hr**"))

# **Plot the Geometry**

In [ ]:
pv.set_jupyter_backend('ipygany')
tank_model.display()